In [1]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
  sys.path.insert(0, module_path)
# endregion   #
# --------------- #

In [2]:
from examples.cleanup.env import Cleanup
from examples.cleanup.agents import Agent
from examples.RPG.utils import load_config
from gem.models.grid_cells import positional_embedding
from gem.models.DDQN import doubleDQN
from gem.utils import visual_field_sprite, image_from_array, animate

from IPython.display import clear_output

import argparse
import matplotlib.pyplot as plt
import random

import torch
import numpy as np

cfg = load_config(argparse.Namespace(config='../configs/config.yaml'))

N_AGENTS = 3
agents = []
for i in range(N_AGENTS):
  agents.append(
  Agent(cfg, appearance = cfg.agent.agent.appearance, 
        model = doubleDQN(
        input_size=5220,
        number_of_actions=4,
        lr=0.001,
        gamma=0.97,
        per=False,
        alpha=0.6,
        beta=0.05,
        beta_increment=0.0006,
        capacity=5000,
    )
          )
  )




env = Cleanup(
  cfg, agents
)

In [3]:
import jax
import jax.numpy as jnp

seed = 42
rng = jax.random.PRNGKey(seed)

# Generate random numbers
random_numbers = jax.random.uniform(rng, shape=(5,))

In [4]:
cfg.experiment.epochs = 1000 # override the number of epochs

rewards = []
losses = 0
epsilon = .9

EPOCH_PRINT_FREQ = 100

for epoch in range(cfg.experiment.epochs): # note that the language is not right. epoch is training. episode is the game
    # Reset the environment at the start of each epoch
        env.reset()
        for agent in env.agents:
            agent.reset()
        random.shuffle(agents)

        done = 0 
        turn = 0
        losses = 0
        game_points = 0

        images = []
        if epoch+1 % EPOCH_PRINT_FREQ == 0:
            epsilon = epsilon*0.95

        while not done:

            turn = turn + 1

            entities = env.get_entities_for_transition()
            # Entity transition
            for entity in entities:
                entity.transition(env)

            # Agent transition
            for agent in agents:

                location_code = positional_embedding(agent.location, env, 3, 3)

                # Get current state
                state = np.concatenate([agent.pov(env).flatten(), location_code]).reshape(1, -1)

                if epoch % EPOCH_PRINT_FREQ == 0:
                    _image = visual_field_sprite(env.world)
                    image = image_from_array(_image)
                    images.append(image)

                # Take action based on current state
                action = agent.model.take_action(state, epsilon)

                (reward,
                next_state,
                done_
                ) = agent.transition(env, state, action)

                if turn >= cfg.experiment.max_turns or done_:
                    done = 1

                exp = (1, (state, action, reward, next_state, done))
                agent.episode_memory.append(exp)
                #TODO: decide on memory update procedures
                agent.model.replay_buffer.add(torch.tensor(state), action, reward, torch.tensor(next_state), done)

                game_points += reward

        rewards.append(game_points)
        
        # At the end of each epoch, train as long as the batch size is large enough.
        if epoch > 10:
            loss = agent.model.train_step(batch_size = 64)
            losses += loss
            
        # Calculate the average of the last 100 rewards
        if len(rewards) >= 100:
            avg_last_100_rewards = sum(rewards[-100:]) / 100
        else:
            avg_last_100_rewards = sum(rewards) / len(rewards)
        if epoch % EPOCH_PRINT_FREQ == 0:
            print(f'Epoch: {epoch} - Epsilon: {epsilon} - Losses {losses} - Avg. last 100 rewards: {avg_last_100_rewards}')
            animate(
                images, filename = f"cleanup_epoch{epoch}", folder = f"{cfg.root}/examples/cleanup/data/"
            )
            losses = 0


Epoch: 0 - Epsilon: 0.81 - Losses 0 - Avg. last 100 rewards: 1.0
Epoch: 100 - Epsilon: 0.7290000000000001 - Losses 49737.19140625 - Avg. last 100 rewards: 11.42
Epoch: 200 - Epsilon: 0.6561000000000001 - Losses 4904.1123046875 - Avg. last 100 rewards: 9.92
Epoch: 300 - Epsilon: 0.5904900000000002 - Losses 4094.6083984375 - Avg. last 100 rewards: 11.68
Epoch: 400 - Epsilon: 0.5314410000000002 - Losses 2669.710693359375 - Avg. last 100 rewards: 7.83
Epoch: 500 - Epsilon: 0.47829690000000014 - Losses 8777.142578125 - Avg. last 100 rewards: 8.64
Epoch: 600 - Epsilon: 0.43046721000000016 - Losses 9319.1318359375 - Avg. last 100 rewards: 7.23
